# ProtoNN in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of ProtoNN. We are using the USPS dataset. Please refer to `fetch_usps.py` and `process_usps.py`for more details on downloading the dataset.

In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import sys
import os
import numpy as np
import tensorflow as tf

from edgeml.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml.graph.protoNN import ProtoNN
import edgeml.utils as utils
import helpermethods as helper

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is placed in the `./usps10` subdirectory.

In [2]:
# Load data
DATA_DIR = './usps10'
train, test = np.load(DATA_DIR + '/train.npy'), np.load(DATA_DIR + '/test.npy')
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]

numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 1)
numClasses = int(numClasses)

y_train = helper.to_onehot(y_train, numClasses)
y_test = helper.to_onehot(y_test, numClasses)
dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]

# Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable `GAMMA`. If `GAMMA` is set to `None`, median heuristic will be used to estimate a good value of $\gamma$ through the `helper.getGamma()` method. This method also returns the corresponding `W` and `B` matrices which should be used to initialize ProtoNN (as is done here).

In [3]:
PROJECTION_DIM = 60
NUM_PROTOTYPES = 60
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.05
NUM_EPOCHS = 200
BATCH_SIZE = 32
GAMMA = 0.0015

In [4]:
W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [5]:
# Setup input and train protoNN
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=10)



Epoch:   0 Batch:   0 Loss: 5.85158 Accuracy: 0.03125
Epoch:   1 Batch:   0 Loss: 1.53823 Accuracy: 0.65625
Epoch:   2 Batch:   0 Loss: 0.81371 Accuracy: 0.87500
Epoch:   3 Batch:   0 Loss: 0.51246 Accuracy: 0.87500
Epoch:   4 Batch:   0 Loss: 0.41875 Accuracy: 0.93750
Epoch:   5 Batch:   0 Loss: 0.36797 Accuracy: 0.96875
Epoch:   6 Batch:   0 Loss: 0.32868 Accuracy: 0.96875
Epoch:   7 Batch:   0 Loss: 0.30316 Accuracy: 0.96875
Epoch:   8 Batch:   0 Loss: 0.29075 Accuracy: 0.96875
Epoch:   9 Batch:   0 Loss: 0.28370 Accuracy: 0.96875
Test Loss: 0.50615 Accuracy: 0.89497
Epoch:  10 Batch:   0 Loss: 0.28014 Accuracy: 0.96875
Epoch:  11 Batch:   0 Loss: 0.27734 Accuracy: 0.96875
Epoch:  12 Batch:   0 Loss: 0.27511 Accuracy: 0.96875
Epoch:  13 Batch:   0 Loss: 0.27126 Accuracy: 0.96875
Epoch:  14 Batch:   0 Loss: 0.26776 Accuracy: 0.96875
Epoch:  15 Batch:   0 Loss: 0.26506 Accuracy: 0.96875
Epoch:  16 Batch:   0 Loss: 0.26371 Accuracy: 0.96875
Epoch:  17 Batch:   0 Loss: 0.26249 Accuracy:

Epoch: 143 Batch:   0 Loss: 0.17900 Accuracy: 0.96875
Epoch: 144 Batch:   0 Loss: 0.17877 Accuracy: 0.96875
Epoch: 145 Batch:   0 Loss: 0.17863 Accuracy: 0.96875
Epoch: 146 Batch:   0 Loss: 0.17844 Accuracy: 0.96875
Epoch: 147 Batch:   0 Loss: 0.17825 Accuracy: 1.00000
Epoch: 148 Batch:   0 Loss: 0.17808 Accuracy: 1.00000
Epoch: 149 Batch:   0 Loss: 0.17798 Accuracy: 1.00000
Test Loss: 0.42054 Accuracy: 0.92288
Epoch: 150 Batch:   0 Loss: 0.17788 Accuracy: 1.00000
Epoch: 151 Batch:   0 Loss: 0.17773 Accuracy: 1.00000
Epoch: 152 Batch:   0 Loss: 0.17753 Accuracy: 1.00000
Epoch: 153 Batch:   0 Loss: 0.17743 Accuracy: 1.00000
Epoch: 154 Batch:   0 Loss: 0.17736 Accuracy: 1.00000
Epoch: 155 Batch:   0 Loss: 0.17724 Accuracy: 1.00000
Epoch: 156 Batch:   0 Loss: 0.17721 Accuracy: 1.00000
Epoch: 157 Batch:   0 Loss: 0.17721 Accuracy: 1.00000
Epoch: 158 Batch:   0 Loss: 0.17713 Accuracy: 1.00000
Epoch: 159 Batch:   0 Loss: 0.17701 Accuracy: 1.00000
Test Loss: 0.41836 Accuracy: 0.92337
Epoch: 1

# Model Evaluation

In [6]:
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
# W, B, Z are tensorflow graph nodes
W, B, Z, _ = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList, expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy 0.92526156
Model size constraint (Bytes):  78240
Number of non-zeros:  19560
Actual model size:  78240
Actual non-zeros:  16488
